<a href="https://colab.research.google.com/github/ryanma5/PL-Project/blob/main/PL_5_year_net_spend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Libraries and mount drive
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Header configuration for identification to avoid Transfermarkt block
#www.whatismybrowser.com to check your browser's user agent
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
}

# Scrape transfermarkt transfer activity table

In [ ]:
print("Scraping data from transfermarkt")

# Get HTML code from page
response = requests.get('https://www.transfermarkt.us/premier-league/fuenfjahresvergleich/wettbewerb/GB1', headers=headers)
if response.status_code != 200:
    print("Failed to retrieve data")

# Parse HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on page
tables = soup.find_all('table')  # This returns a list of all <table> elements

# Retrieve first table from page (first table is always league table)
if tables:
    transfer_activity_table = tables[0]
else:
    print(f"No tables found on the page for {season_url}")

Scraping data from transfermarkt


# Function for parsing transfer activity table and populating a list that stores team names, and their net spend last 5 years

In [ ]:
print("Parsing data from transfermarkt")

# Find all rows in the table (skip the header row)
rows = transfer_activity_table.find_all('tr')[2:]  # Skip the header row and footer row

# Initialize an empty dictionary to store clubs and their net spend
club_net_spend = {}

# Loop through each row and extract the relevant data (club name, net spend)
for row in rows:
    # Extract club name from the second column
    team_cell = row.find_all('td')[1]  # Second `<td>` is for the club name
    team = team_cell.find('a').text.strip() if team_cell and team_cell.find('a') else team_cell.text.strip()

    # Extract net spend from the third column
    net_spend_cell = row.find_all('td')[-1]  # Third `<td>` is for the net spend
    net_spend_text = net_spend_cell.text.strip().replace("€", "").replace(",", "").replace("m", "") # Retain only numbers string

    # Convert to string to numerical value
    try:
        net_spend = float(net_spend_text)  # Direct conversion after cleanup
    except ValueError:
        print(f"Unable to parse net spend: {net_spend_text} for team {team}, defaulting to 0.")
        net_spend = 0

    # Add the club name and net spend to the dictionary
    club_net_spend[team] = net_spend

# Print the resulting dictionary
print(club_net_spend)


Parsing data from transfermarkt
{'Arsenal FC': -556.46, 'Aston Villa': -257.67, 'AFC Bournemouth': -142.17, 'Brentford FC': -113.65, 'Brighton & Hove Albion': -59.41, 'Chelsea FC': -940.76, 'Crystal Palace': -173.96, 'Everton FC': 32.84, 'Fulham FC': -146.47, 'Ipswich Town': -132.77, 'Leicester City': -26.98, 'Liverpool FC': -299.35, 'Manchester City': -169.43, 'Manchester United': -650.44, 'Newcastle United': -434.91, 'Nottingham Forest': -262.54, 'Southampton FC': -71.53, 'Tottenham Hotspur': -544.58, 'West Ham United': -328.35, 'Wolverhampton Wanderers': -17.53}


In [ ]:
# Convert the accumulated team points into a DataFrame
team_net_spend_df = pd.DataFrame(list(club_net_spend.items()), columns=['Club', 'Net Spend'])

# Sort the DataFrame by Total Points in descending order (optional)
team_net_spend_df = team_net_spend_df.sort_values(by='Net Spend', ascending=False)

# Display the final DataFrame
print(team_net_spend_df)


                       Club  Net Spend
7                Everton FC      32.84
19  Wolverhampton Wanderers     -17.53
10           Leicester City     -26.98
4    Brighton & Hove Albion     -59.41
16           Southampton FC     -71.53
3              Brentford FC    -113.65
9              Ipswich Town    -132.77
2           AFC Bournemouth    -142.17
8                 Fulham FC    -146.47
12          Manchester City    -169.43
6            Crystal Palace    -173.96
1               Aston Villa    -257.67
15        Nottingham Forest    -262.54
11             Liverpool FC    -299.35
18          West Ham United    -328.35
14         Newcastle United    -434.91
17        Tottenham Hotspur    -544.58
0                Arsenal FC    -556.46
13        Manchester United    -650.44
5                Chelsea FC    -940.76


In [ ]:
# Convert df to csv and save
team_net_spend_df.to_csv('/content/drive/My Drive/Colab Notebooks/PL_5years_netspend.csv', index=False) # your desired location